#### 一. 概览
操作系统检测到一个混乱的 TCP 连接(`Disrupted TCP connection`)通常需要很长的一段时间(linux上默认配置为11分钟); `AMQP 0-9-1`提供了一种心跳机制,从而实现: 
* 让应用层可以迅速的发现 TCP 连接混乱
* 抵御网络设备关闭空闲的 TCP 连接(一段时间没有收发消息)    
   某些网络工具(HAProxy)和设备可能会主动关闭空闲的 TCP 连接(如果发现一段时间内连接上没有信息传输), 因此使用 heartbeat 还能避免客户端和 mq 的连接不会被关闭. 设置了 heartbeat timeout 为30秒的连接, 会每15秒产生一次消息传输 

`TCP keepalives`是 TCP 栈的特性, 实现了类似的功能. `TCP keepalives` 可以和 `Heartbeat` 配合使用

#### 二. Heartbeat Timeout Value  
Client 在和 Server 端建立连接时, 会尝试商讨出一个`heartbeat timeout`值, 这个值取 Client 端和 Server 端两者之中的最小值; 这个值默认是60秒 

* `heartbeat timeout`值定义了在经过多长时间后, 被监控的 `RabbitMQ` 和客户端之间的 TCP 连接不可达
* `heartbeat timeout = 60` 默认值, 单位是秒; 如果将这个值配置成0, 说明客户端不开启心跳


#### 三. Heartbeat 框架
&nbsp;&nbsp;&nbsp;&nbsp;Heartbeat框架每隔`heartbeat_timeout / 2`秒发送一次, 这个值也被叫做`heartbeat interval`. 当丢失2个 heartbeat 后
, 就会认为对端不可达. 当 client 端发现 RabbitMQ 节点不可达后, 需要重新连接(re-connect);  
&nbsp;&nbsp;&nbsp;&nbsp;所有的通信( protocal operations; publish msg; consume ack)都会被当做一次有效的 heartbeat ;Client端可以选择无视这些通信自顾自的发送心跳, 或者只在必要的时候才发送心跳

#### 四. 关闭/开启heartbeat
1. `关闭heartbeat`    
   当把间隔时间设置为0时, 客户端的 heartbeat 就会被关闭. 初始设置了一个很低的`TCP keepalive`值作为'非活动连接'检测, 否则关闭 heartbeat 时极其不推荐的, 尤其是对于 producer 而言, 不设置 heartbeat 间隔时间将引起很大风险   
   
2. `开启heartbeat`  
   因为server端会有一个默认的非零 heartbeat 事件, 因此客户端配置的 heartbeat 要想生效, 必须必 Server 端的值小
   ```java
   ConnectionFactory cf = new ConnectionFactory();

   // set the heartbeat timeout to 60 seconds
   cf.setRequestedHeartbeat(60);
   ```
   
#### 五. 太小的 heartbeat 超时时间会导致`false positive`
&nbsp;&nbsp;&nbsp;&nbsp;`heartbeat timeout`设置的太小会导致`false positive`: 即误把对端认成不可达, 其实对端可达. 原因主要有: 短暂的网络拥塞, 短连接控制等; 小于5秒的 `heartbeat timeout` 值更可能引起 `false positive`; 如果设置成1秒, 则极有可能发生`false positive`; 大多数场景下, 这个值一般设置成`5~20秒`是最优的

#### 六. Tcp Keepalives
`Tcp Keepalives` 是操作系统层面的 heartbeat; 选择使用 Tcp Keepalives 来进行连接检测的主要好处是: 所有运行在这台机器上的 TCP 连接将使用同一个超时值.   
和 Rabbitmq 相关的系统配置可以在[这里](https://www.rabbitmq.com/networking.html)找到

